# 14_딥러닝(2)
## - MNIST (필기체숫자) 인식 -

MNIST - 개요

- => MNIST
- MNIST (Modified National Institute of Standards and Technology)는 손으로 직접 쓴 숫자(필기체 숫자)들로 이루어진 데이터 셋 (Data Set) 이며,
- 우리가 새로운 프로그래밍 언어를 배울 때 'Hello, World'를 출력하는 것처럼, MNIST는 딥러닝을 배울 때 반드시 거쳐야 하는 'Hello, World' 같은 존재임.
- MNIST는 0부터 9까지의 숫자 이미지로 구성되며, 60000개의 트레이닝 데이터와 10000개 테스트 데이터로 이루어져 있음.

MNIST - 구조 (1)
- => mnist_train.csv
- mnist_train.csv 파일에는 학습에 이용될 수 있도록 정답(lable)이 있는 총 60000개의 데이터 존재함. 1개의 데이터는 785개의 숫자가 콤마(,)로 분리 되어 있는데, 정답을 나타내는 1개의 숫자와 실제 필기체 숫자 이미지를 나타내는 784개의 숫자로 구성되어 잇음.

- => mnist_test.csv
- mnist_test.csv 파일에는 총 10000개의 데이터가 있으며, 학습을 마친 후에 구현된 딥러닝 아키텍처가 얼마나 잘 작동하는지 테스트 하기 위해 사용됨. 테스트 데이터 또한 정답(lable)이 포함된 785 개의 숫자로 되어 있음.

MNIST - 구조 (2)

In [ ]:
# MNIST 가져오기 (numpy.loadtxt 활용)
# training data
import numpy as np

training_data = np.loadtxt('mnist_train.csv', delimiter = ',', dtype = np.float32)
test_data = np.loadtxt('mnist_test.csv', delimiter = ',', dtype = np.float32)

print("training_data.shape = ", training_data.shape, "test_data.shape = ", test_data.shape)

MNIST - 구조 (3)

In [ ]:
# training_data 이미지 표현
import matplotlib.pyplot as plt

img = training_data[0][1:].reshape(28, 28)

plt.imshow(img, cmap = 'gray')
plt.show()

NeuralNetwork class - MNIST (필기체 숫자) 인식 클래스

- external function  
  
- def sigmoid(x): # 0 또는 1을 출력하기 위한 sigmoid 함수  

- def numerical_derivative(f, x): # 수치미분함수  

- NeuralNetwork class

- class NeuralNetwork:
    def __init__(self, gate_name, xdata, tdata) # 입력/정답 데이터/가중치/바이어스 초기화  
    def feed_forward(self) # feed forward 이용하여 손실함수 값 계산  
    def error_val(self) # 손실함수 값 계산(외부 출력을 위해 사용됨)  
    def train(self) # 수치미분을 이용하여 손실함수 최소값 찾는 method  
    def predict(self, xdata) # 미래 값 예측 method  

- usage
- 입력노드 784개, 은닉노드 100개, 출력노드 10개의 NeuralNetwork 객체 생성
- nn = NeuralNetwork(784, 100, 10)

- for step in range(30001): # 60000개의 training data 중 50 % 데이터로 학습 진행
    index = np.random.randint(0, 59999) # 60000개의 데이터 가운데 random 하게 30000개 선택
    nn.train(training_data[index]) # random 하게 선택된 training_data를 이용하여 학습 진행

구현코드 - MNIST (필기체 숫자 인식)

In [ ]:
# [1] external function (sigmoid, numerical_derivative)
import numpy as np

# sigmoid 함수
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 수치미분 함수
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags = ['multi_index'], op_flags = ['readwrite'])
    
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x + delta_x)
        
        x[idx] = tmp_val - delta_x
        fx2 = f(x) # f(x - delta_x)
        grad[idx] = (fx1 - fx2) / (2 * delta_x)
        
        x[idx] = tmp_val
        it.iternext()
        
    return grad

In [ ]:
# [2] NeuralNetwork (__init__, feed_forward)
class NeuralNetwork:
    
    # 생성자
    def __init__(self, input_nodes, hidden_nodes, output_nodes):
        
        self.__input_nodes = input_nodes   # input_nodes = 784
        self.__hidden_nodes = hidden_nodes # hidden_nodes = 1004
        self.__output_nodes = output_nodes # output_nodes = 10
        
        # 2층 hidden layer unit
        # 가중치 W, 바이어스 b 초기화
        self.__W2 = np.random.rand(self.__input_nodes, self.__hidden_nodes) # W2 = (784 X 100)
        self.__b2 = np.random.rand(self.__hidden_nodes) # b2 = (100,)
        
        # 3층 output layer unit
        self.__W3 = np.random.randn(self.__hidden_nodes, self.__output_nodes) # W3 = (100 X 10)
        self.__b3 = np.random.rand(self.__output_nodes) # b3 = (10,)
        
        # 학습률 learning rate 초기화
        self.__learning_rate = 1e-4
        
    # feed forward를 이용하여 입력층에서부터 출력층까지 데이터를 전달하고 손실 함수 값 계산
    # loss_val(self) 메소드와 동일한 코드. loss_val(self)은 외부 출력용으로 사용됨
    def feed_forward(self):
        
        delta = 1e-7 # log 무한대 발산 방지
        
        z1 = np.dot(self.input_data, self.__W2) + self.__b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.__W3) + self.__b3
        y = sigmoid(z2)
        
        # cross-entropy
        return -np.sum(self.target_data * np.log(y + delta) + (1 - self.target_data) * np.log((1 - y) + delta))

# [2] NeuralNetwork (loss_val)
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7 # log 무한대 발산 방지
        
        z1 = np.dot(self.input_data, self.__W2) + self.__b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.__W3) + self.__b3
        y = sigmoid(z2)
        
        # cross-entropy
        return -np.sum(self.target_data * np.log(y + delta) + (1 - self.target_data) * np.log((1 - y) + delta))

# [2] NeuralNetwork (train, predict)
# input_data : 784 개, target_data : 10개
    def train(self, training_data):
        
        # normalize
        self.target_data = np.zeros(self.__output_nodes) + 0.01
        self.target_data[int(training_data[0])] = 0.99
        
        self.input_data = (training_data[1:] / 255.0 * 0.99) + 0.01
        
        f = lambda x : self.feed_forward()
        
        self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
        self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
        self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
        self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)
        
    # query. 즉 미래 값 예측 함수
    def predict(self, input_data):
        
        z1 = np.dot(input_data, self.__W2) + self.__b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.__W3) + self.__b3
        y = sigmoid(z2)
        
        predicted_num = np.argmax(y)
        
        return predicted_num
    
    # NeuralNetwork (accuracy)
    # 정확도 측정함수
    def accuracy(self, test_dat):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_data)):
            
            label = int(test_data[index, 0])
            
            # normalize
            data = (test_data[index, 1:] / 255.0 * 0.99) + 0.01
            
            predicted_num = self.predict(data)
            
            if label == predicted_num:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        print("Current Accuracy = ", 100 * (len(matched_list) / (len(test_data))), "%")
        
        return matched_list, not_matched_list

검증코드 - MNIST (필기체 숫자 인식)

In [ ]:
# [3] usage (data feeding, learning process)
input_nodes = 784
hidden_nodes = 100
output_nodes = 10

nn = NeuralNetwork(input_nodes, hidden_nodes, output_nodes)

for step in range(30001): # 전체 training data 중 50 %
    
    # 총 60,000개의 training data 가운데 random 하게 30,000개 선택
    index = np.random.randint(0, len(training_data) - 1)
    
    nn.train(training_data[index])
    
    if step % 400 == 0:
        print("step =", step, ", loss_val", nn.loss_val())

In [ ]:
# accuray 계산

nn.accuracy(test_data)